# Day 12: Hot Springs

I'm going to do a slightly different format going forward and use this as a place to record thoughts.

Part one of todays question is about finding the number of ways a pattern of numbers can fit into a partially known, partially unknown line.
I've been playing a lot of nonnograms and this is effectively that but with only 1 axis.

In [6]:
example = [
    "???.### 1,1,3",
    ".??..??...?##. 1,1,3",
    "?#?#?#?#?#?#?#? 1,3,1,6",
    "????.#...#... 4,1,1",
    "????.######..#####. 1,6,5",
    "?###???????? 3,2,1"
]

examplepartonesolution = 21

21

There's a bunch of math to find the number of permutations of a given sequence in a given space but it doesn't work well with the additional constraints so most of this math is unused.

Given the length of a run and a known space the number of possibilities is:
$$
possibilities = space - run + 1
$$

given a series of numbers, instead of a single number the minimum space they occupy is

$$
minlength = (\sum^{n}_{i=1} run_{i} + 1) - 1
$$

This assumes a gap of 1 between each number in the sequence, you can then use the minlength to compute the possibilities of this configuration.
However any of the gaps can grow if there's space
Reading some more theory on this it seems as though to distribute v extra spaces amonst n numbers you would get

$$
possibilities = {n-2+v \choose n-2}
$$

you would need to consider this for all possibilities

$$
    \sum^{extraspaces}_{v=0} {n-2+v \choose n-2}
$$

In [16]:
## compute the min length
function minlength(runlengths)
    if length(runlengths) == 0
        return 0
    end
    return sum(runlengths) + length(runlengths) - 1
end

## Compute the possibilites, while this is cool math it isn't useful
function possibilities(space, runlengths)
    n = length(runlengths)
    extraspaces = space - minlength(runlengths)
    binomial(n + extraspaces, n)
end

println(possibilities(4, (4)))
println(possibilities(4, (2)))
println(possibilities(4, (2, 1)))
println(possibilities(5, (2, 1)))
println(possibilities(15, (1, 3, 1, 6)))
println(possibilities(15, (1, 1, 1, 1, 1, 1, 1)))

1
3
1
3
5
36


It's easy to write a function to check if a configuration matches a given constraint

In [3]:
function meetsconstraints(constraint, configuration)
    all(
        c -> c[1] == '?' || c[1] == c[2],
        zip(constraint, configuration)
    )
end

meetsconstraints(['#', '?', '.', '?'], ['#', '#', '.', '.'])

minlength (generic function with 1 method)

generating all the possibile valid configuration for a line could also be done recursively
By combining this with the `meetsconstraints` function you get a reasonable solution to part one that first generates all possible permutations and then eliminates ones that aren't possible.

In [21]:
using Memoization

@memoize Dict function possibleconfigurations(constraint, runlengths, leadingworkingspring)
    valid(c) = meetsconstraints(constraint, c)
    space = length(constraint)
    minspace = minlength(runlengths)

    if minspace == 0
        if space == 0
            return [[]]
        end
        allspaces = fill('.', space)
        return valid(allspaces) ? [allspaces] : []
    end

    if leadingworkingspring
        if constraint[begin] == '.' || constraint[begin] == '?'
            return filter(
                valid,
                map(c -> ['.', c...], possibleconfigurations(constraint[begin+1:end], runlengths, false))
            )
        else
            return []
        end
    end

    if minspace > space
        return []
    end

    (run, rest...) = runlengths
    filter(valid, reduce(vcat,
        map(
            function (leadingspace)
                a = vcat(fill('.', leadingspace), fill('#', run))
                map(r -> vcat(a, r), possibleconfigurations(constraint[begin+leadingspace+run:end], [rest...], true))
            end,
            0:(space-minspace)
        );
        init=Array{Char}(undef, 0)
    ))
end

println(possibleconfigurations("????", (4), false))
println(possibleconfigurations("????", (2), false))
println(possibleconfigurations("##??", (2, 1), false))
println(possibleconfigurations("???????????????", (1, 3, 1, 6), false))
println(possibleconfigurations("??????????", (1, 5), false))

Any[Any['#', '#', '#', '#']]
Any[['#', '#', '.', '.'], ['.', '#', '#', '.'], Any['.', '.', '#', '#']]
Any[['#', '#', '.', '#']]
Any[['#', '.', '#', '#', '#', '.', '#', '.', '#', '#', '#', '#', '#', '#', '.'], ['#', '.', '#', '#', '#', '.', '#', '.', '.', '#', '#', '#', '#', '#', '#'], ['#', '.', '#', '#', '#', '.', '.', '#', '.', '#', '#', '#', '#', '#', '#'], ['#', '.', '.', '#', '#', '#', '.', '#', '.', '#', '#', '#', '#', '#', '#'], ['.', '#', '.', '#', '#', '#', '.', '#', '.', '#', '#', '#', '#', '#', '#']]
Any[['#', '.', '#', '#', '#', '#', '#', '.', '.', '.'], ['#', '.', '.', '#', '#', '#', '#', '#', '.', '.'], ['#', '.', '.', '.', '#', '#', '#', '#', '#', '.'], ['#', '.', '.', '.', '.', '#', '#', '#', '#', '#'], ['.', '#', '.', '#', '#', '#', '#', '#', '.', '.'], ['.', '#', '.', '.', '#', '#', '#', '#', '#', '.'], ['.', '#', '.', '.', '.', '#', '#', '#', '#', '#'], ['.', '.', '#', '.', '#', '#', '#', '#', '#', '.'], ['.', '.', '#', '.', '.', '#', '#', '#', '#', '#'], ['.', '.', 

When getting the part two the approach of generating all possibilities and then filtering falls down because there are too many possibilities.
We can effectively combine both algorithms writing one that only generates valid entries and counts as it goes, this will also make it more memory efficient.

In [4]:
using Memoization

@memoize Dict function possiblevalidconfigurations(constraint, runlengths, leadingworkingspring)
   ## no more runs of broken springs left, everything in the constraint has to be unkown or working
   ## if the constraint is empty this will work.
   if length(runlengths) == 0
      return all(v -> v == '?' || v == '.', constraint) ? 1 : 0
   end

   ## not enough space
   if length(constraint) < minlength(runlengths)
      return 0
   end

   ## if there is / could be a leading working spring then the number of permutations need to take into account what would happen if there was
   s1 = if constraint[begin] == '.' || constraint[begin] == '?'
      possiblevalidconfigurations(constraint[begin+1:end], runlengths, false)
   else
      0
   end

   ## if there is / could be a a leading broken spring then the number of permutations need to take into account what would happen if there was
   s2 = if !leadingworkingspring && (constraint[begin] == '#' || constraint[begin] == '?')
      (run, rest...) = runlengths
      ## check if the constraint would allow for the leading broken spring
      if all(v -> v == '?' || v == '#', constraint[begin:begin+run-1])
         possiblevalidconfigurations(constraint[begin+run:end], [rest...], true)
      else
         0
      end
   else
      0
   end

   ## add the 2 possibilities
   return s1 + s2
end

println(possiblevalidconfigurations("???.###", (1, 1, 3), false))
println(possiblevalidconfigurations(".??..??...?##.", (1, 1, 3), false))
println(possiblevalidconfigurations("?#?#?#?#?#?#?#?", (1, 3, 1, 6), false))

1
4
1


Parsing and putting it all together

In [8]:
function parsecontents(contents)
    map(contents) do line
        (constraint, nums) = split(line)
        runlengths = map(d -> parse(Int, d), split(nums, ','))
        (constraint, runlengths)
    end
end

parsecontents(example)

6-element Vector{Tuple{SubString{String}, Vector{Int64}}}:
 ("???.###", [1, 1, 3])
 (".??..??...?##.", [1, 1, 3])
 ("?#?#?#?#?#?#?#?", [1, 3, 1, 6])
 ("????.#...#...", [4, 1, 1])
 ("????.######..#####.", [1, 6, 5])
 ("?###????????", [3, 2, 1])

In [10]:
function partone(contents)
    parsed = parsecontents(contents)
    sum(
        map(parsed) do (constraint, runlengths)
            possiblevalidconfigurations(constraint, runlengths, false)
        end
    )
end

partone(example) == examplepartonesolution

true

## Part Two

For part two we found out that the rows are folded over 5 fold times.

In [12]:
exampleparttwosoluation = 525152

525152

In [13]:
function unfold5(constraint, runlengths)
    (
        constraint * "?" * constraint * "?" * constraint * "?" * constraint * "?" * constraint,
        (runlengths..., runlengths..., runlengths..., runlengths..., runlengths...)
    )
end

unfold5 (generic function with 1 method)

In [14]:
function parttwo(contents)
    parsed = parsecontents(contents)
    unfolded = map(t -> unfold5(t[1], t[2]), parsed)
    return sum(map(unfolded) do (constraint, runlengths)
        possiblevalidconfigurations(constraint, runlengths, false)
    end)
end

parttwo(example) == exampleparttwosoluation

true

## Result

In [15]:
include("./aoc.jl")

println(7402)
println(3384337640277)
execute(12, partone, parttwo)


7402
7402
3384337640277
